#Task 1: Tokenization



In [ ]:
# Download Gutenberg Corpus
# It is the dataset on which this model will be trained.

In [ ]:
import re
import os
import sys
import operator as op
from math import log
from functools import reduce

from ipy_table import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go
import plotly.plotly as py

init_notebook_mode(connected=True)
corpus_directory = 'Gutenberg/txt'

corpus = []
sentences = []
words = []

In [ ]:
def clean_sentence(text):
    text = re.sub(r"[^A-Za-z0-9^,?!.\/'+=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " , ", text)
    text = re.sub(r"\.", " . ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s+", " ", text)

    return text

In [ ]:
def tokenise(sentence):
    sentence = clean_sentence(sentence).split()
    sentences.append(sentence)

    return sentence

def tokenise_corpus(corpus):
    # Convert byte object to string
    corpus = corpus.decode("utf-8")

    # Remove all new line characters, as they may be at abrupt places.
    # Also change case to lower.
    corpus = re.sub(r'\n', " ", corpus).lower()

    # Place new line characters at the end of sentences.
    # Ignoring abbreviations for the sake of simplicity.
    corpus = re.sub(r"([.!?])", r"\1\n", corpus)

    # Split corpus into sentences.
    corpus = corpus.split('\n')

    return [tokenise(sentence) for sentence in corpus]

In [ ]:
# Obtain list of all files in Gutenberg Corpus.
files = os.listdir(corpus_directory)

In [ ]:
# Load data from all files in Gutenberg Corpus.
for file in files[:5]:
    with open(os.path.join(corpus_directory, file), 'rb') as f:
        contents = f.read()
        corpus += tokenise_corpus(contents)

In [ ]:
# Test Cell, loads data only from first file.
# with open(os.path.join(corpus_directory, files[0]), 'rb') as f:
#     contents = f.read()
#     corpus += tokenise_corpus(contents)

In [ ]:
# Get a count of all unique words in the corpus 
def token_counts(corpus):
    tokens = {}
    for sentence in corpus:
        for word in sentence:
            if word not in tokens:
                tokens[word] = 0
            tokens[word] += 1
    return tokens

tokens = token_counts(corpus)
sorted_tokens = sorted(tokens.items(), key = lambda x: x[1], reverse=True)

frequencies = []

for pair in sorted_tokens:
    words.append(pair[0])
    frequencies.append(pair[1])

# Create a trace
trace = go.Scatter(
    x = words,
    y = frequencies
)

data = [trace]

iplot(data, filename='basic-line')

#Language Model and Smoothing

In [ ]:
class LanguageModel(object):
    def __init__(self, units, n=4):
        self.units = units
        self.n = n
        self.n_grams = [{}]
        self.vocab_size = [0]
        self.seen_counts = [0]

    def ncr(self, n, r):
        r = min(r, n-r)
        numer = reduce(op.mul, range(n, n-r, -1), 1)
        denom = reduce(op.mul, range(1, r+1), 1)
        return numer//denom

    def get_n_grams(self, n):
        n_grams = {}

        for unit in self.units:
            unit_len = len(unit)

            for i in range(unit_len - n):
                n_gram = tuple(unit[i:i+n])
                if n_gram not in n_grams:
                    n_grams[n_gram] = 0
                n_grams[n_gram] += 1

        if n == 1: self.vocab_size.append(len(n_grams))
        else: self.vocab_size.append(self.ncr(self.vocab_size[1], n-1))

        self.n_grams.append(n_grams)
        self.seen_counts.append(len(n_grams))

    def get_sorted_n_grams(self, n):
        if n > self.n:
            print('Max permitted value of n is:', self.n)
            return

        return sorted(self.n_grams[n].items(), key = lambda x: x[1], reverse=True)

    def split_grams_freq(self, sorted_n_grams):
        n_grams = []
        frequencies = []

        for pair in sorted_n_grams:
            # n-grams are in the form of tuples for the sake of simplicity.
            if isinstance(pair[0], tuple):
                n_grams.append(' '.join(pair[0]))

            frequencies.append(pair[1])
        
        return n_grams, frequencies

    def run(self):
        for i in range(1, self.n+1):
            self.get_n_grams(i)

    def plot_graph(self, n_grams, frequencies, topk=None, loglog=False):
        if loglog:
            n_grams = [i for i in range(len(frequencies))]
            layout = dict(title = 'Frequency counts of various n-grams',
                          xaxis = dict(title = 'n-grams', type='log'),
                          yaxis = dict(title = 'frequency', type='log'))
        else:
            layout = dict(title = 'Frequency counts of various n-grams',
                          xaxis = dict(title = 'n-grams'),
                          yaxis = dict(title = 'frequency'))

        # Create a trace
        trace = go.Scatter(
            x = n_grams[:topk],
            y = frequencies[:topk],
            name = 'n grams',
            line = dict(color = ('rgb(205, 12, 24)'),
                        width = 4))

        data = [trace]

        fig = dict(data=data, layout=layout)
        iplot(fig, filename='styled-line')

    '''
    Plot ngrams in the descending order of frequency
    '''
    def plot_n_gram_freqs(self, n, topk=None, loglog=False):
        if n > self.n:
            print('Max permitted value of n is:', self.n)
            return

        sorted_n_grams = self.get_sorted_n_grams(n)
        n_grams, frequencies = self.split_grams_freq(sorted_n_grams)

        self.plot_graph(n_grams, frequencies, topk, loglog=loglog)

    '''
    Given (N-1) gram, and the value 'N',
        1. Print the possibilities that complete the n-gram
        2. Plot them in decresing order of frequency
    '''
    def complete_ngram(self, n_gram, n, topk=None):
        if n > self.n:
            print('Max permitted value of n is:', self.n)
            return

        if not isinstance(n_gram, tuple):
            if isinstance(n_gram, str):
                n_gram = n_gram.split()

            n_gram = tuple(n_gram)

        sorted_n_grams = self.get_sorted_n_grams(n)

        n_grams = []
        frequencies = []

        for pair in sorted_n_grams:
            if n_gram == pair[0][:n-1]:
                n_grams.append(' '.join(pair[0]))
                frequencies.append(pair[1])

        for i in range(min(10, len(n_grams))):
            print(n_grams[i], frequencies[i])

        self.plot_graph(n_grams, frequencies, topk)

    def laplace_smoothing(self, n_grams, alpha=0.001):
        n = len(n_grams[0])
        score = 1

        for n_gram in n_grams:
            # Numerator of the score
            count_n = alpha
            if n_gram in self.n_grams[n]: count_n += self.n_grams[n][n_gram]

            # Denominator of the score
            count_d = alpha*self.vocab_size[n]
            if n_gram[:-1] in self.n_grams[n-1]: count_d += self.n_grams[n-1][n_gram[:-1]]

            '''
            Uncomment the line below to view the disparity in data.
            Most of the possiblen-grams are never seen in corpus.
            '''
            # print(count_n, count_d - alpha*self.vocab_size[n], self.vocab_size[n])
            score *= (count_n/count_d)

        return score

    def backoff(self, scores):
        weights = [[1],
                   [0.39, 0.61],
                   [0.025, 0.275, 0.700]]

        ind = len(scores)-1
        return [sum([a*b for a, b in zip(weights[ind], scores)])]

    def kneser_ney(self, n_grams, delta=0.5, epsilon=1e-7):
        n = len(n_grams[0])
        score = 1

        for n_gram in n_grams:
            # Numerator of the score
            count_n = 0
            if n_gram in self.n_grams[n]: count_n += self.n_grams[n][n_gram] - delta

            # Denominator of the score
            count_d = 0
            pwi = 0
            for key in self.n_grams[n].keys():
                if n_gram[:-1] == key[:-1]:
                    count_d += self.n_grams[n][key]

                if n_gram[-1] == key[-1]:
                    pwi += self.n_grams[n][key]

            if count_d == 0:
                return 0

            pwi /= self.seen_counts[n]
            lambdai = delta/count_d
            # print(pwi, lambdai, count_n)

            score *= ((count_n/count_d) + pwi*lambdai + epsilon)

        return score

    def score_sequence(self, sequence, alpha=0.001, scoring_function='laplace_smoothing', backoff=False):
        seq_len = len(sequence)

        combined_n_grams = []
        scores = []

        for n in range(1, self.n+1):
            n_grams = []

            for i in range(seq_len-n+1):
                n_grams.append(tuple(sequence[i:i+n]))
                if i == seq_len-n:
                    break

            if len(n_grams) > 0: combined_n_grams.append(n_grams)

        # Only giving bigrams and above to scoring functions
        for n_grams in combined_n_grams[1:]:
            if scoring_function == 'laplace_smoothing':
                scores.append(self.laplace_smoothing(n_grams, alpha=alpha))

            elif scoring_function == 'kneser_ney':
                scores.append(self.kneser_ney(n_grams))
                if scores[-1] == 0:
                    break

                scores = [scores[-1]]

        if backoff: scores = self.backoff(scores)

        return sum(scores)


#Task 2: Spelling Correction

In [ ]:
# Create a Character Level Language Model
class CharLM(LanguageModel):
    def __init__(self, units, n=4):
        LanguageModel.__init__(self, units, n)

    def suggest_spelling(self, word):
        suggestions = {}
        suggestions[word] = self.score_sequence(word)

        word = list(word)
        length = len(word)

        def insert(suggestions, word, score):
            if word not in suggestions:
                if len(suggestions) < 10:
                    suggestions[word] = score
                else:
                    min_key = min(suggestions, key=suggestions.get)
                    if score > suggestions[min_key]:
                        del suggestions[min_key]
                        suggestions[word] = score

        for i in range(length):
            # Remove one character
            new_word_r = ''.join(word[:i] + word[i+1:])
            score_r = self.score_sequence(new_word_r)

            insert(suggestions, new_word_r, score_r)

            for char in self.n_grams[1]:
                # Add one character
                new_word_a = ''.join(word[:i] + list(char) + word[i:])
                score_a = self.score_sequence(new_word_a)

                insert(suggestions, new_word_a, score_a)

                # Swap one character
                for j in range(length-1):
                    new_word_r = list(new_word_r)
                    new_word_s = ''.join(new_word_r[:j] + list(char) + new_word_r[j:])
                    score_s = self.score_sequence(new_word_s)

                    insert(suggestions, new_word_s, score_s)

        if str(word) in suggestions: print('Your spelling\'s perfect!')
        else: print('Did you mean:\n', [str(key) for key in suggestions.keys()])

char_lm = CharLM(words, n=4)
char_lm.run()

In [ ]:
# Plot unigram frequencies
char_lm.plot_n_gram_freqs(2, topk=25)
char_lm.plot_n_gram_freqs(2, topk=25, loglog=True)

In [ ]:
# Give a correctness score to word spelling.
sequence = 'hat'

print('Laplace Smoothing                 :', char_lm.score_sequence(sequence))
print('Laplace Smoothing with backoff    :', char_lm.score_sequence(sequence, backoff=True))
print('Kneser Ney Smoothing              :', char_lm.score_sequence(sequence, scoring_function='kneser_ney'))

print('\n')
if sequence not in words:
    char_lm.suggest_spelling(sequence)
print('\n')

sequence = 'phat'

print('Laplace Smoothing                 :', char_lm.score_sequence(sequence))
print('Laplace Smoothing with backoff    :', char_lm.score_sequence(sequence, backoff=True))
print('Kneser Ney Smoothing              :', char_lm.score_sequence(sequence, scoring_function='kneser_ney'))

print('\n')
if sequence not in words:
    char_lm.suggest_spelling(sequence)
print('\n')

# print(char_lm.seen_counts)

#Task 3 : Grammaticality Test

In [ ]:
# Create a Word Level Language Model
class WordLM(LanguageModel):
    def __init__(self, units, n=4):
        LanguageModel.__init__(self, units, n)

word_lm = WordLM(sentences, n=4)
word_lm.run()

In [ ]:
# Plot n-gram frequencies
word_lm.plot_n_gram_freqs(2)
word_lm.plot_n_gram_freqs(2, loglog=True)

In [ ]:
word_lm.complete_ngram('and', 2)

In [ ]:
# Give a correctness score to sentence grammaticality.
sequence = 'I am a man.'

sequence = tokenise(sequence.lower())
print('Laplace Smoothing                 :', word_lm.score_sequence(sequence, alpha=0.00001))
print('Laplace Smoothing with backoff    :', word_lm.score_sequence(sequence, alpha=0.00001, backoff=True))
print('Kneser Ney Smoothing              :', word_lm.score_sequence(sequence, scoring_function='kneser_ney'))
print('\n')

sequence = 'man I am a.'

sequence = tokenise(sequence.lower())
print('Laplace Smoothing                 :', word_lm.score_sequence(sequence, alpha=0.00001))
print('Laplace Smoothing with backoff    :', word_lm.score_sequence(sequence, alpha=0.00001, backoff=True))
print('Kneser Ney Smoothing              :', word_lm.score_sequence(sequence, scoring_function='kneser_ney'))
print('\n')

# print(word_lm.seen_counts)